In [36]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [37]:
import pandas as pd
import requests
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert
from shapely.geometry import Point
import overpy
import ast
from decimal import Decimal
from pandasgui import show
import zipfile

from script import geolocaliser
from script import process_data
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [38]:
import geopandas as gpd

# Chemin vers le fichier .shp
shapefile_path = "data/communes_cotieres/communes_cotieres.shp"

# Charger le shapefile dans un GeoDataFrame
df_cotieres = gpd.read_file(shapefile_path)

df_cotieres = df_cotieres[['code', 'nom', 'NumDep']].drop_duplicates().reset_index(drop=True)

# Afficher les 5 premières lignes
show(df_cotieres)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_34540\4120728332.py:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_34540\4120728332.py:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_34540\4120728332.py:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel

1. Création du fichier des communes cotières

In [39]:
# # URL du fichier Excel
# url_cotier = 'https://comersis.com/free-download.php?f=communes_littorales_2019.xlsx&d=littoral'

# # Télécharger le fichier
# response = rq.get(url_cotier)

# # Lire le fichier Excel dans un DataFrame
# df_cotieres = pd.read_excel(BytesIO(response.content))
# df_cotieres = df_cotieres[df_cotieres['Région'] != "Départements d'Outre-Mer"]


# print(df_cotieres.columns)

In [40]:
df_cotieres = df_cotieres.rename(columns={'code': 'code_commune', 
                                          'nom': 'Nom commune', 
                                          'NumDep': 'departement'})

liste_cotieres = df_cotieres['Nom commune'].unique().tolist()
print(liste_cotieres[1:10])

region = pd.read_csv('data/code_region.csv',sep=";",encoding="utf8")

df_cotieres = df_cotieres.merge(
    region,  # Utilisation du DataFrame region ici
    left_on='departement', 
    right_on='departmentCode'  # Vérifie le nom exact de la colonne dans region
)

['Hézo', 'Île-aux-Moines', "Île-d'Arz", 'Séné', 'Surzur', 'Vannes', 'Bono', 'Marseille', 'Bonifacio']


In [41]:
url = "https://www.insee.fr/fr/statistiques/fichier/7739582/ensemble.zip"
zip_path = "ensemble.zip"
extracted_folder = "ensemble"

# Télécharger le fichier ZIP
response = requests.get(url)
with open(zip_path, 'wb') as file:
    file.write(response.content)

# Dézipper le fichier ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# Charger le fichier CSV dans un DataFrame
csv_path = os.path.join(extracted_folder, "donnees_communes.csv")
df_pop = pd.read_csv(csv_path,sep=";",encoding="utf8")

os.remove("ensemble.zip")

df_pop = df_pop[['COM',"PMUN"]].rename(columns={'COM':'code_commune',
                                                'PMUN':'Population'})

# Afficher les premières lignes du DataFrame
print(df_pop.head())

df_cotieres = df_cotieres.merge(df_pop,left_on='code_commune',right_on='code_commune')


  code_commune  Population
0        01001         832
1        01002         267
2        01004       14854
3        01005        1897
4        01006         113


In [42]:
stats_region_filtered = df_cotieres.groupby('regionName').apply(
    lambda x: pd.Series({
        'nombre_communes': x['Nom commune'].nunique(),
        'population_totale': x['Population'].sum()  # Décommente si la colonne 'Population' existe
    })
).reset_index()

# Afficher les résultats
tableau_communes = stats_region_filtered.sort_values(by='nombre_communes', ascending=False).fillna("").style.hide(axis="index")

# Afficher le tableau sans NaN et sans index
tableau_communes

C:\Users\flori\AppData\Local\Temp\ipykernel_34540\3091059007.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



regionName,nombre_communes,population_totale
Bretagne,290,1423714
Normandie,238,1067474
Nouvelle-Aquitaine,199,1325337
Pays de la Loire,80,984537
Occitanie,74,837192
Hauts-de-France,60,438188
Provence-Alpes-Côte d'Azur,49,1043236


In [43]:
# Calcul des statistiques par commune
stats_communes = df_cotieres.groupby('Nom commune').apply(
    lambda x: pd.Series({
        'Population': x['Population'].sum()  # Décommente si la colonne 'Population' existe
    })
).reset_index()

top_20_communes = (
    stats_communes
    .sort_values(by='Population', ascending=False)
    .head(20)
    .style.hide(axis="index")
    .format({'Population': lambda x: f"{x:,.0f}".replace(',', ' ')})  # Remplacement de la virgule par un espace
)


top_20_communes


C:\Users\flori\AppData\Local\Temp\ipykernel_34540\296672310.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Nom commune,Population
Nantes,323 204
Bordeaux,261 804
Toulon,180 452
Havre,166 058
Brest,139 619
Perpignan,119 656
Rouen,114 083
Caen,108 200
Arles,100 830
Dunkerque,86 788


In [44]:
# Charger le fichier GeoJSON des communes de France depuis le lien GitHub
url_commune = 'https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/communes.geojson'
response_commune = requests.get(url_commune)
communes_geojson = response_commune.json()

# Charger le GeoJSON dans un GeoDataFrame
gdf_communes = gpd.GeoDataFrame.from_features(communes_geojson['features'])
gdf_communes_cotieres = gdf_communes[gdf_communes['nom'].isin(liste_cotieres)]
gdf_communes_cotieres = gdf_communes_cotieres.merge(df_cotieres[['Nom commune', 'Population']], 
                                                    left_on='nom', 
                                                    right_on='Nom commune'
                                                    ).drop(columns=['Nom commune'])


In [45]:
# Calculer les centroïdes pour chaque commune
gdf_communes_cotieres['centroid'] = gdf_communes_cotieres.geometry.centroid
gdf_communes_cotieres['latitude_centre'] = gdf_communes_cotieres['centroid'].y
gdf_communes_cotieres['longitude_centre'] = gdf_communes_cotieres['centroid'].x
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['centroid'])

liste_commune = gdf_communes_cotieres['nom'].to_list()

On utilise l'API Overpass pour successivement géolocaliser les mairies, les stations de transport et les plages de la commune.

In [46]:
# Appliquer la fonction sur chaque commune
api = overpy.Overpass()

gdf_communes_cotieres[['latitude_mairie', 'longitude_mairie']] = gdf_communes_cotieres.apply(
    lambda row: pd.Series(geolocaliser.get_townhall_coordinates(row['nom'], api)),
    axis=1
)

Dans le cas où Overpass ne permet pas de géolocaliser la mairie, on fait une recherche par mot-clé à partir de l'API Adresse de l'Institut National Géographique (ING)

In [49]:
import importlib
importlib.reload(geolocaliser)

# Liste des mots-clés pour localiser les mairies
mots_cles = [
    "Mairie+de", 
    "Hotel+de+ville+de", 
    "Bureau+du+Maire+de", 
    "Salle+des+Fêtes+de", 
    "Maison+Communale+de", 
    "Centre+Administratif+de", 
    "Hôtel+de+Ville", 
    "Bâtiment+Municipal+de", 
    "Mairie+municipale+de", 
    "Maison+des+Services+de"
]

# Compléter les coordonnées manquantes
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles, 
    colonne_latitude='latitude_mairie', 
    colonne_longitude='longitude_mairie'
)

Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Île-d'Arz
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Île-d'Arz
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Île-d'Arz
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Bono
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Bono
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Bono
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Braud-et-Saint-Louis
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Braud-et-Saint-Louis
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Braud-et-Saint-Louis
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Braud-et-Saint-Louis
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Tabanac
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Tabanac
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Tabanac
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Tourne
A

In [50]:
# Liste des mots-clés pour localiser les plages
mots_cles_plages = [ 
    "Plage+de",
    "Plage"
]

# Compléter les coordonnées manquantes pour les plages
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_plages, 
    colonne_latitude='latitude_plage', 
    colonne_longitude='longitude_plage'
)

Aucune donnée trouvée pour l'adresse : Plage+Portets
Aucune donnée trouvée pour l'adresse : Plage+Créhen
Aucune donnée trouvée pour l'adresse : Plage+Saint-Lormel
Aucune donnée trouvée pour l'adresse : Plage+Asques
Aucune donnée trouvée pour l'adresse : Plage+Épine
Aucune donnée trouvée pour l'adresse : Plage+de+Beaussais-sur-Mer
Aucune donnée trouvée pour l'adresse : Plage+Beaussais-sur-Mer
Aucune donnée trouvée pour l'adresse : Plage+de+Cancale
Aucune donnée trouvée pour l'adresse : Plage+de+Rozel
Aucune donnée trouvée pour l'adresse : Plage+de+Saillans
Aucune donnée trouvée pour l'adresse : Plage+Aizier
Aucune donnée trouvée pour l'adresse : Plage+Réville
Aucune donnée trouvée pour l'adresse : Plage+Vains
Aucune donnée trouvée pour l'adresse : Plage+Tracy-sur-Mer
Aucune donnée trouvée pour l'adresse : Plage+de+Camoël
Aucune donnée trouvée pour l'adresse : Plage+Crach
Aucune donnée trouvée pour l'adresse : Plage+de+Étel
Aucune donnée trouvée pour l'adresse : Plage+Étel
Aucune donnée 

In [51]:
# Pour les plages
api = overpy.Overpass()

gdf_communes_cotieres['beach_coordinates'] = gdf_communes_cotieres.apply(
    lambda row: geolocaliser.get_beach_coordinates(row['nom'], api),
    axis=1
)

In [52]:
# Pour les stations de transport
api = overpy.Overpass()

gdf_communes_cotieres['station'] = gdf_communes_cotieres.apply(
    lambda row: geolocaliser.get_station_coordinates(row['nom'], api),
    axis=1
)

# Liste des mots-clés pour localiser les gares
mots_cles_gares = [ 
    "Gare+de"
]

# Compléter les coordonnées manquantes pour les pgares
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_gares, 
    colonne_latitude='latitude_gare', 
    colonne_longitude='longitude_gare'
)

In [53]:
# Utilisation de la fonction avec un GeoDataFrame
api = overpy.Overpass()

gdf_communes_cotieres[['latitude_port', 'longitude_port']] = gdf_communes_cotieres.apply(
    lambda row: pd.Series(geolocaliser.get_ports(row['nom'], api)),
    axis=1
)

# Liste des mots-clés pour localiser les plages
mots_cles_ports = [ 
    "Port+de",
]

# Compléter les coordonnées manquantes pour les plages
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_ports, 
    colonne_latitude='latitude_port', 
    colonne_longitude='longitude_port'
)

Aucune donnée trouvée pour l'adresse : Port+de+Heuqueville
Aucune donnée trouvée pour l'adresse : Port+de+Mauny
Aucune donnée trouvée pour l'adresse : Port+de+Plogoff
Aucune donnée trouvée pour l'adresse : Port+de+Conquet
Aucune donnée trouvée pour l'adresse : Port+de+Noyelles-sur-Mer
Aucune donnée trouvée pour l'adresse : Port+de+Saint-Armel
Aucune donnée trouvée pour l'adresse : Port+de+Floirac


In [54]:
# Application
gdf_communes_cotieres['station'] = gdf_communes_cotieres['station'].apply(process_data.fix_coordinates_format)
gdf_communes_cotieres['beach_coordinates'] = gdf_communes_cotieres['beach_coordinates'].apply(process_data.fix_coordinates_format)

In [55]:
gdf_communes_cotieres["beach_coordinates"] = gdf_communes_cotieres.apply(
    lambda row: row["beach_coordinates"] + [(row["latitude_plage"], row["longitude_plage"])] if row["latitude_plage"] is not None and row["longitude_plage"] is not None else row["beach_coordinates"],
    axis=1
)

In [56]:
# Calcul de la part de chaque variable renseignée
summary = {
    "Variable": gdf_communes_cotieres.columns,
    "Part (%)": [
        gdf_communes_cotieres[col].notnull().mean() * 100 
        for col in gdf_communes_cotieres.columns
    ]
}

# Création d'un DataFrame pour présentation
summary_df = pd.DataFrame(summary)

# Filtrer uniquement les colonnes désirées
columns_to_display = [
    "geometry", "Population", "latitude_mairie", 
    "beach_coordinates", "station", "latitude_port"
]
filtered_summary_df = summary_df[summary_df["Variable"].isin(columns_to_display)]

# Style pour affichage dans le notebook
styled_table = (
    filtered_summary_df.style
    .hide(axis="index")  # Cache l'index
    .set_caption("Part de variables renseignées")
    .format({"Part (%)": "{:.1f}%"})  # Formate les pourcentages
)

# Affichage dans le notebook
styled_table

Variable,Part (%)
geometry,100.0%
Population,100.0%
latitude_mairie,90.7%
beach_coordinates,100.0%
station,100.0%
latitude_port,58.1%


In [57]:
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['latitude_plage','longitude_plage','latitude_gare', 'longitude_gare','code'])

In [58]:
gdf_communes_cotieres.to_csv('data/communes_cotieres.csv',encoding="utf8",sep=";",index=False)